In [3]:
import pandas as pd
import numpy as np
from ekonlpy.tag import Mecab

In [4]:
df_20 = pd.read_csv(r'C:\Users\haech\my_workspace\prac_scrapy\df_20_preprocessed.csv')

In [5]:
con_ls = list(df_20['content'].dropna())

In [6]:
len(con_ls)

38544

In [94]:
# mecab 토큰화
mecab = Mecab()
con_noun = []
for c in con_ls:
    nouns = mecab.nouns(c)
    con_noun.append(nouns)

In [96]:
len(con_noun)

38544

In [98]:
mecab_df = pd.DataFrame(pd.Series(con_noun))

In [100]:
mecab_df.to_csv('mecab_tokens_nouns.csv', index=False)

In [101]:
# Word2Vec embedding
from gensim.models import Word2Vec
embedding_model = Word2Vec(con_noun, size=100, window = 3, min_count=10, workers=4, iter=100, sg=1) # min_count=50, 50번 미만 등장 제외

In [104]:
# check embedding result
embedding_model.most_similar(positive=["라이브"], topn=20)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('바자회', 0.6762439012527466),
 ('기도회', 0.4164663255214691),
 ('선입', 0.41581711173057556),
 ('랜', 0.41353949904441833),
 ('스타트', 0.4108400344848633),
 ('시청', 0.4093373119831085),
 ('중계', 0.4072938859462738),
 ('출연', 0.4067833721637726),
 ('공개', 0.4065122604370117),
 ('사군자', 0.406261146068573),
 ('작성자', 0.40540811419487),
 ('문법', 0.399910569190979),
 ('박람회', 0.3992534279823303),
 ('정탑', 0.3980051279067993),
 ('송가', 0.39384451508522034),
 ('금순', 0.3936668336391449),
 ('지킴', 0.39204320311546326),
 ('치독', 0.38996797800064087),
 ('현금가', 0.38928020000457764),
 ('학사모', 0.38722774386405945)]

In [50]:
model_vec = embedding_model.wv

In [51]:
len(model_vec.vectors)

9107

In [52]:
from sklearn.manifold import TSNE

In [53]:
tsne = TSNE(random_state=0)
tsne_points = tsne.fit_transform(model_vec.vectors)

In [54]:
tsne_df = pd.DataFrame(tsne_points, columns=['x_coord', 'y_coord'])
tsne_df['word'] = [model_vec.index2word[i] for i in tsne_df.index]

In [56]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

Loading BokehJS ...

In [57]:
# prepare the data in a form suitable for bokeh.
plot_data = ColumnDataSource(tsne_df)

# create the plot and configure it
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 1000,
                   plot_height =1000,
                   active_scroll='wheel_zoom',
                   output_backend="webgl"
                  )

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = '@word') )

tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='orange')

# adjust visual elements of the plot
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# show time!
show(tsne_plot);